In [ ]:
#リストを読み込んだトラカンデータから、距離案分を行う

In [2]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd
import re

In [5]:
f = open('exec_list.txt', 'r', encoding='UTF-8')

sepre = re.compile(r'\-+')
parre = re.compile(r'([^\s]+?)\s*?=(.*)')
block_num = 0
blocks = {}

# lines = f.readlines()
for line in f:

    if sepre.match(line):
        block_num += 1
    
    # print (line, end='')
    if block_num not in blocks:
        blocks.setdefault(block_num, {})

    m = parre.match(line)
    if m is None:
        continue

    key = m.group(1)
    val = m.group(2)

    values = blocks[block_num]
    values[key] = eval(val)
    blocks[block_num] = values

f.close()

# ブロックごとに何かする
finddatas = 1
itemid = 20
for idx in blocks:
    block = blocks[idx]
    if len(block) <= 1:
        continue
    if block.get("start_osm") is None:
        continue
    print("------------------------------------------------------")
    for k in block:
        print("%s => %s" % (k, block[k]))
    #if block["start_osm"] == 311923788:
    #    finddatas = 1
    #    continue
    if finddatas == 1:
        itemid = itemid + 1
        start_osm = block["start_osm"]
        mid_osm = block["mid_osm"]
        end_osm =  block["end_osm"]
        start_mesh = block["start_mesh"]
        end_mesh = block["end_mesh"]
        strids = "1_" + str(itemid)
        connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")
        getdbname = "kddi_00_new_average"
        dbname = "kddi_trkn00_monthly_anbun_" + strids
        trknName = "trkn_00_grp"
        sttime = "2022-04-01 00:00:00"
        getdbname6 = "kddi_06_new_average"
        dbname6 = "kddi_trkn06_monthly_anbun_" + strids
        trknName6 = "trkn_06_grp"
        sttime6 = "2022-04-01 06:00:00"
        getdbname12 = "kddi_12_new_average"
        dbname12 = "kddi_trkn12_monthly_anbun_" + strids
        trknName12 = "trkn_12_grp"
        sttime12 = "2022-04-01 12:00:00"
        getdbname18 = "kddi_18_new_average"
        dbname18 = "kddi_trkn18_monthly_anbun_" + strids
        trknName18 = "trkn_18_grp"
        sttime18 = "2022-04-01 18:00:00"
        datedir = "230301"
        f = open(datedir+'/'+ dbname + '.csv', 'w')
        
        dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
        
        for j in range(24):
            sttimestr = dte.strftime('%H:%M:%S')
            eddte = dte + datetime.timedelta(minutes=15) 
            edtimestr = eddte.strftime('%H:%M:%S')
            
            #特定日付のKDDIメッシュ情報を全取得
            cur =connection.cursor()
            sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
            sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
            sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
            cur.execute(sql)
            kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
            #cur.close()
        
            one_target_list = []
            one_target_list.append(start_mesh)
            one_target_list.append(end_mesh)
            two_target_list = []
            two_target_list.append(start_osm)
            if len(mid_osm) != 0:
                for target in mid_osm:
                    two_target_list.append(target)
            if start_osm != end_osm:
                two_target_list.append(end_osm)
        
            #cur =connection.cursor()
            sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
            cur.execute(sql)
            for row in cur:
                stlat = row[0]
                stlng = row[1]
            sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
            cur.execute(sql)
            for row in cur:
                edlat = row[0]
                edlng = row[1]
            #cur.close()
        
            if stlat > edlat:
                wklat = stlat
                stlat = edlat
                edlat = wklat
            if stlng > edlng:
                wklng = stlng
                stlng = edlng
                edlng = wklng
        
            #print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))
        
            keylist = []
            cnt = 0
            for item in two_target_list:
                sql = "SELECT a.grid_code,a.latitude,a.longitude"
                sql += " FROM pointdata a, roadmap r"
                sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
                sql += " order by ST_Distance(a.geom, r.geom);"
                #cur =connection.cursor()
                cur.execute(sql)
                childlist = []
                for row in cur:
                    if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                        childlist.append([row[0], row[1], row[2]])
                    #print(row[0])
                worklist = [item, childlist]
                keylist.append(worklist)
                cnt = cnt + 1
                #cur.close()
            #特定日付のトラカンデータを全取得 > 緯度経度
            #cur =connection.cursor()
            #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
            #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
            #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
            #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
            #sql += "ORDER BY trakanmap.gridcode"
            sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
            sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
            sql += "WHERE times >=  '"+sttimestr+"' AND times < '"+edtimestr+"' AND times < '"+edtimestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
            sql += "ORDER BY mesh"
            #print(sql)
            cur.execute(sql)
            trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
            cur.close()
            trknDf
            #keylist
        
            from geopy.distance import geodesic
        
            #データが存在する道路IDごとに交通量を平均化して取得する
            lstItems = []
            cntDt = 0
            wks = ""
            wknp=""
            find = 0
        
            trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
            trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

            stlen = len(trknStParam)
            
            trknSt = 0
            if stlen == 0:
                print(str(start_mesh) + " mesh empty!")
                dte = dte + datetime.timedelta(minutes=15) 
                continue
            else:
                for m in range(stlen):
                    trknSt = trknSt + trknStParam['danmen'].iloc[m]
                    #print(str(trknStParam['danmen'].iloc[m]))

            edlen = len(trknEdParam)
            trknEd = 0
            if edlen == 0:
                print(str(end_mesh) + " mesh empty!")
                dte = dte + datetime.timedelta(minutes=15) 
                continue
            else:
                for m in range(edlen):
                    trknEd = trknEd + trknEdParam['danmen'].iloc[m]
        
            st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
            ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]
        
            dis_km = geodesic(st_station, ed_station)
            if dis_km == 0:
                dte = dte + datetime.timedelta(minutes=15) 
                continue
        
            #cur =connection.cursor()
            for row in keylist:
                for meshid in row[1]:
                    #路線のメッシュ数分ループ
                    #print(str(meshid[0]))
                    kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                    if len(kddiParam) == 0:
                        print(str(meshid[0]) + " is kddi none!")
                        continue
                    #路線のメッシュがkddiに存在したなら
                    
                    kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                    kddi_km = geodesic(st_station, kddi_station)
                    
                    if trknSt < trknEd:
                        ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                    else:
                        ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                    #print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                    #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                    f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                    #cur.execute(sql)
                    #connection.commit()
        
            #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
            f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
            #cur.execute(sql)
            #connection.commit()
            #cur.close()
            dte = dte + datetime.timedelta(minutes=15) 
            #print(lstItems)
        f.close()
        
        print("faze:2")
        
        f = open(datedir+'/'+ dbname6 + '.csv', 'w')
        
        dte = datetime.datetime.strptime(sttime6, '%Y-%m-%d %H:%M:%S')
        
        for j in range(24):
            sttimestr = dte.strftime('%H:%M:%S')
            eddte = dte + datetime.timedelta(minutes=15) 
            edtimestr = eddte.strftime('%H:%M:%S')
            #特定日付のKDDIメッシュ情報を全取得
            cur =connection.cursor()
            sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname6 + " "
            sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
            sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
            cur.execute(sql)
            kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
            #cur.close()
        
            one_target_list = []
            one_target_list.append(start_mesh)
            one_target_list.append(end_mesh)
            two_target_list = []
            two_target_list.append(start_osm)
            if len(mid_osm) != 0:
                for target in mid_osm:
                    two_target_list.append(target)
            if start_osm != end_osm:
                two_target_list.append(end_osm)
        
            #cur =connection.cursor()
            sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
            cur.execute(sql)
            for row in cur:
                stlat = row[0]
                stlng = row[1]
            sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
            cur.execute(sql)
            for row in cur:
                edlat = row[0]
                edlng = row[1]
            #cur.close()
        
            if stlat > edlat:
                wklat = stlat
                stlat = edlat
                edlat = wklat
            if stlng > edlng:
                wklng = stlng
                stlng = edlng
                edlng = wklng
        
            #print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))
        
            keylist = []
            cnt = 0
            for item in two_target_list:
                sql = "SELECT a.grid_code,a.latitude,a.longitude"
                sql += " FROM pointdata a, roadmap r"
                sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
                sql += " order by ST_Distance(a.geom, r.geom);"
                #cur =connection.cursor()
                cur.execute(sql)
                childlist = []
                for row in cur:
                    if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                        childlist.append([row[0], row[1], row[2]])
                    #print(row[0])
                worklist = [item, childlist]
                keylist.append(worklist)
                cnt = cnt + 1
                #cur.close()
            #特定日付のトラカンデータを全取得 > 緯度経度
            #cur =connection.cursor()
            #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
            #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
            #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
            #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
            #sql += "ORDER BY trakanmap.gridcode"
            sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName6  + " "
            sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
            sql += "WHERE times >=  '"+sttimestr+"' AND times < '"+edtimestr+"' AND times < '"+edtimestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
            sql += "ORDER BY mesh"
            #print(sql)
            cur.execute(sql)
            trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
            cur.close()
            trknDf
            #keylist
        
            from geopy.distance import geodesic
        
            #データが存在する道路IDごとに交通量を平均化して取得する
            lstItems = []
            cntDt = 0
            wks = ""
            wknp=""
            find = 0
        
            trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
            trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

            stlen = len(trknStParam)

            trknSt = 0
            if stlen == 0:
                print(str(start_mesh) + " mesh empty!")
                dte = dte + datetime.timedelta(minutes=15) 
                continue
            else:
                for m in range(stlen):
                    trknSt = trknSt + trknStParam['danmen'].iloc[m]
        
            edlen = len(trknEdParam)
            trknEd = 0
            if edlen == 0:
                print(str(end_mesh) + " mesh empty!")
                dte = dte + datetime.timedelta(minutes=15) 
                continue
            else:
                for m in range(edlen):
                    trknEd = trknEd + trknEdParam['danmen'].iloc[m]

            st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
            ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]
        
            dis_km = geodesic(st_station, ed_station)
            if dis_km == 0:
                dte = dte + datetime.timedelta(minutes=15) 
                continue
        
            #cur =connection.cursor()
            for row in keylist:
                for meshid in row[1]:
                    #路線のメッシュ数分ループ
                    #print(str(meshid[0]))
                    kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                    if len(kddiParam) == 0:
                        print(str(meshid[0]) + " is kddi none!")
                        continue
                    #路線のメッシュがkddiに存在したなら
                    
                    kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                    kddi_km = geodesic(st_station, kddi_station)
                    
                    if trknSt < trknEd:
                        ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                    else:
                        ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                    #print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                    #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                    f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                    #cur.execute(sql)
                    #connection.commit()
        
            #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
            f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
            #cur.execute(sql)
            #connection.commit()
            #cur.close()
            dte = dte + datetime.timedelta(minutes=15) 
            #print(lstItems)
        f.close()
        
        print("faze:3")
        
        f = open(datedir+'/'+ dbname12 + '.csv', 'w')
        
        dte = datetime.datetime.strptime(sttime12, '%Y-%m-%d %H:%M:%S')
        
        for j in range(24):
            sttimestr = dte.strftime('%H:%M:%S')
            eddte = dte + datetime.timedelta(minutes=15) 
            edtimestr = eddte.strftime('%H:%M:%S')
            #特定日付のKDDIメッシュ情報を全取得
            cur =connection.cursor()
            sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname12 + " "
            sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
            sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
            cur.execute(sql)
            kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
            #cur.close()
        
            one_target_list = []
            one_target_list.append(start_mesh)
            one_target_list.append(end_mesh)
            two_target_list = []
            two_target_list.append(start_osm)
            if len(mid_osm) != 0:
                for target in mid_osm:
                    two_target_list.append(target)
            if start_osm != end_osm:
                two_target_list.append(end_osm)
        
            #cur =connection.cursor()
            sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
            cur.execute(sql)
            for row in cur:
                stlat = row[0]
                stlng = row[1]
            sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
            cur.execute(sql)
            for row in cur:
                edlat = row[0]
                edlng = row[1]
            #cur.close()
        
            if stlat > edlat:
                wklat = stlat
                stlat = edlat
                edlat = wklat
            if stlng > edlng:
                wklng = stlng
                stlng = edlng
                edlng = wklng
        
            #print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))
        
            keylist = []
            cnt = 0
            for item in two_target_list:
                sql = "SELECT a.grid_code,a.latitude,a.longitude"
                sql += " FROM pointdata a, roadmap r"
                sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
                sql += " order by ST_Distance(a.geom, r.geom);"
                #cur =connection.cursor()
                cur.execute(sql)
                childlist = []
                for row in cur:
                    if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                        childlist.append([row[0], row[1], row[2]])
                    #print(row[0])
                worklist = [item, childlist]
                keylist.append(worklist)
                cnt = cnt + 1
                #cur.close()
            #特定日付のトラカンデータを全取得 > 緯度経度
            #cur =connection.cursor()
            #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
            #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
            #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
            #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
            #sql += "ORDER BY trakanmap.gridcode"
            sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName12  + " "
            sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
            sql += "WHERE times >=  '"+sttimestr+"' AND times < '"+edtimestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
            sql += "ORDER BY mesh"
            #print(sql)
            cur.execute(sql)
            trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
            cur.close()
            trknDf
            #keylist
        
            from geopy.distance import geodesic
        
            #データが存在する道路IDごとに交通量を平均化して取得する
            lstItems = []
            cntDt = 0
            wks = ""
            wknp=""
            find = 0
        
            trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
            trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

            stlen = len(trknStParam)
            trknSt = 0
            if stlen == 0:
                print(str(start_mesh) + " mesh empty!")
                dte = dte + datetime.timedelta(minutes=15) 
                continue
            else:
                for m in range(stlen):
                    trknSt = trknSt + trknStParam['danmen'].iloc[m]
        
            edlen = len(trknEdParam)
            trknEd = 0
            if edlen == 0:
                print(str(end_mesh) + " mesh empty!")
                dte = dte + datetime.timedelta(minutes=15) 
                continue
            else:
                for m in range(edlen):
                    trknEd = trknEd + trknEdParam['danmen'].iloc[m]
        
            st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
            ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]
        
            dis_km = geodesic(st_station, ed_station)
            if dis_km == 0:
                dte = dte + datetime.timedelta(minutes=15) 
                continue
        
            #cur =connection.cursor()
            for row in keylist:
                for meshid in row[1]:
                    #路線のメッシュ数分ループ
                    #print(str(meshid[0]))
                    kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                    if len(kddiParam) == 0:
                        print(str(meshid[0]) + " is kddi none!")
                        continue
                    #路線のメッシュがkddiに存在したなら
                    
                    kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                    kddi_km = geodesic(st_station, kddi_station)
                    
                    if trknSt < trknEd:
                        ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                    else:
                        ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                    #print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                    #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                    f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                    #cur.execute(sql)
                    #connection.commit()
        
            #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
            f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
            #cur.execute(sql)
            #connection.commit()
            #cur.close()
            dte = dte + datetime.timedelta(minutes=15) 
            #print(lstItems)
        f.close()
        
        print("faze:4")
        
        f = open(datedir+'/'+ dbname18 + '.csv', 'w')
        
        dte = datetime.datetime.strptime(sttime18, '%Y-%m-%d %H:%M:%S')
        
        for j in range(24):
            sttimestr = dte.strftime('%H:%M:%S')
            eddte = dte + datetime.timedelta(minutes=15) 
            edtimestr = eddte.strftime('%H:%M:%S')

            #特定日付のKDDIメッシュ情報を全取得
            cur =connection.cursor()
            sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname18 + " "
            sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
            sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
            cur.execute(sql)
            kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
            #cur.close()
        
            one_target_list = []
            one_target_list.append(start_mesh)
            one_target_list.append(end_mesh)
            two_target_list = []
            two_target_list.append(start_osm)
            if len(mid_osm) != 0:
                for target in mid_osm:
                    two_target_list.append(target)
            if start_osm != end_osm:
                two_target_list.append(end_osm)
        
            #cur =connection.cursor()
            sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
            cur.execute(sql)
            for row in cur:
                stlat = row[0]
                stlng = row[1]
            sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
            cur.execute(sql)
            for row in cur:
                edlat = row[0]
                edlng = row[1]
            #cur.close()
        
            if stlat > edlat:
                wklat = stlat
                stlat = edlat
                edlat = wklat
            if stlng > edlng:
                wklng = stlng
                stlng = edlng
                edlng = wklng
        
            #print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))
        
            keylist = []
            cnt = 0
            for item in two_target_list:
                sql = "SELECT a.grid_code,a.latitude,a.longitude"
                sql += " FROM pointdata a, roadmap r"
                sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
                sql += " order by ST_Distance(a.geom, r.geom);"
                #cur =connection.cursor()
                cur.execute(sql)
                childlist = []
                for row in cur:
                    if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                        childlist.append([row[0], row[1], row[2]])
                    #print(row[0])
                worklist = [item, childlist]
                keylist.append(worklist)
                cnt = cnt + 1
                #cur.close()
            #特定日付のトラカンデータを全取得 > 緯度経度
            #cur =connection.cursor()
            #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
            #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
            #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
            #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
            #sql += "ORDER BY trakanmap.gridcode"
            sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName18  + " "
            sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
            sql += "WHERE times >=  '"+sttimestr+"' AND times < '"+edtimestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
            sql += "ORDER BY mesh"
            #print(sql)
            cur.execute(sql)
            trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
            cur.close()
            trknDf
            #keylist
        
            from geopy.distance import geodesic
        
            #データが存在する道路IDごとに交通量を平均化して取得する
            lstItems = []
            cntDt = 0
            wks = ""
            wknp=""
            find = 0
        
            trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
            trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

            stlen = len(trknStParam)
            trknSt = 0
            if stlen == 0:
                print(str(start_mesh) + " mesh empty!")
                dte = dte + datetime.timedelta(minutes=15) 
                continue
            else:
                for m in range(stlen):
                    trknSt = trknSt + trknStParam['danmen'].iloc[m]
        
            edlen = len(trknEdParam)
            trknEd = 0
            if edlen == 0:
                print(str(end_mesh) + " mesh empty!")
                dte = dte + datetime.timedelta(minutes=15) 
                continue
            else:
                for m in range(edlen):
                    trknEd = trknEd + trknEdParam['danmen'].iloc[m]
        
            st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
            ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]
        
            dis_km = geodesic(st_station, ed_station)
            if dis_km == 0:
                dte = dte + datetime.timedelta(minutes=15) 
                continue
        
            #cur =connection.cursor()
            for row in keylist:
                for meshid in row[1]:
                    #路線のメッシュ数分ループ
                    #print(str(meshid[0]))
                    kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                    if len(kddiParam) == 0:
                        print(str(meshid[0]) + " is kddi none!")
                        continue
                    #路線のメッシュがkddiに存在したなら
                    
                    kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                    kddi_km = geodesic(st_station, kddi_station)
                    
                    if trknSt < trknEd:
                        ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                    else:
                        ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                    #print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                    #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                    f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                    #cur.execute(sql)
                    #connection.commit()
        
            #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
            f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
            #cur.execute(sql)
            #connection.commit()
            #cur.close()
            dte = dte + datetime.timedelta(minutes=15) 
            #print(lstItems)
        f.close()
        connection.close()


------------------------------------------------------
start_osm => 333602850
end_osm => 266396724
mid_osm => [333608604, 334023275, 332179492, 334023280, 334023282, 334023276, 458864946, 458864947, 332179489, 266396726, 458864950, 458864949, 332286381]
start_mesh => 50325578243
end_mesh => 50325662142
50325569431 is kddi none!
50325660331 is kddi none!
50325660331 is kddi none!
50325660323 is kddi none!
50325660331 is kddi none!
50325660314 is kddi none!
50325660242 is kddi none!
50325569431 is kddi none!
50325660331 is kddi none!
50325660331 is kddi none!
50325660323 is kddi none!
50325660331 is kddi none!
50325660314 is kddi none!
50325660242 is kddi none!
50325569431 is kddi none!
50325660331 is kddi none!
50325660331 is kddi none!
50325660323 is kddi none!
50325660331 is kddi none!
50325660314 is kddi none!
50325660242 is kddi none!
50325569431 is kddi none!
50325660331 is kddi none!
50325660331 is kddi none!
50325660323 is kddi none!
50325660331 is kddi none!
50325660314 is kddi 

In [6]:
print("complete")

complete
